<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/Tokeninzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install transformers
!pip install datasets

In [2]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from datasets import Dataset, load_dataset

In [3]:
dataset = load_dataset('drive/MyDrive', data_files={
    'train': 'nikluge-sc-2023-train.jsonl',
    'validation': 'nikluge-sc-2023-dev.jsonl',
    'test': 'nikluge-sc-2023-test.jsonl'
})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
model_checkpoint = "skt/kogpt2-base-v2"

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token = '<sep>', padding_side='left')
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
sentence1 = []
sentence3 = []

for dictionary in dataset['train']['input']:
    sentence1.append(dictionary["sentence1"])
    sentence3.append(dictionary["sentence3"])

In [6]:
dataset['train'] = dataset['train'].add_column('sentence1', sentence1)
dataset['train'] = dataset['train'].add_column('sentence3', sentence3)

In [7]:
dataset['train']

Dataset({
    features: ['id', 'input', 'output', 'sentence1', 'sentence3'],
    num_rows: 120140
})

In [8]:
sentence1_val = []
sentence3_val = []

for dictionary in dataset['validation']['input']:
    sentence1_val.append(dictionary["sentence1"])
    sentence3_val.append(dictionary["sentence3"])

dataset['validation'] = dataset['validation'].add_column('sentence1', sentence1_val)
dataset['validation'] = dataset['validation'].add_column('sentence3', sentence3_val)

sentence1_test = []
sentence3_test = []

for dictionary in dataset['test']['input']:
    sentence1_test.append(dictionary["sentence1"])
    sentence3_test.append(dictionary["sentence3"])

dataset['test'] = dataset['test'].add_column('sentence1', sentence1_test)
dataset['test'] = dataset['test'].add_column('sentence3', sentence3_test)

In [ ]:
# Define a function to tokenize a single example
def tokenize_example(example):
    first_tokens = tokenizer.tokenize(example["input"]["sentence1"])
    third_tokens = tokenizer.tokenize(example["input"]["sentence3"])
    target_tokens = tokenizer.tokenize(example["output"])

    # Add special tokens to the tokenized sentences
    first_tokens = [start_token1] + first_tokens + [end_token1]
    third_tokens = [] + third_tokens + [end_token3]
    target_tokens = [start_target_token] + target_tokens + [end_target_token]

    # Combine the tokens into a single input sequence
    input_tokens = first_tokens + third_tokens

    # Convert tokens to input IDs
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    target_ids = tokenizer.convert_tokens_to_ids(target_tokens)

    return {
        "input_tokens": input_tokens,
        "input_ids": input_ids,
        "labels" : labels
    }

# Tokenize the entire dataset
tokenized_dataset = dataset.map(tokenize_example)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(dataset["train"].column_names)

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_tokens', 'input_ids', 'target_tokens', 'target_ids'],
        num_rows: 120140
    })
    validation: Dataset({
        features: ['input_tokens', 'input_ids', 'target_tokens', 'target_ids'],
        num_rows: 15017
    })
    test: Dataset({
        features: ['input_tokens', 'input_ids', 'target_tokens', 'target_ids'],
        num_rows: 15018
    })
})

In [9]:
def tokenize_and_format_example(example):
    first_sentence = example["input"]["sentence1"]
    third_sentence = example["input"]["sentence3"]
    target_sentence = example["output"]

    # Tokenize the input sentences
    input_tokens = tokenizer(
        '</s>'+first_sentence+'<sep>'+third_sentence+'</s>',
        padding="max_length",
        max_length=512,  # Adjust this as needed
        truncation=True,
        return_tensors="pt",  # Return PyTorch tensors
    )

    # Tokenize the target sentence
    target_tokens = tokenizer(
        target_sentence,
        padding="max_length",
        max_length=128,  # Adjust this as needed
        truncation=True,
        return_tensors="pt",  # Return PyTorch tensors
    )

    # Create a dictionary with input and target sequences
    formatted_example = {
        "input_ids": input_tokens["input_ids"].squeeze(),
        "attention_mask": input_tokens["attention_mask"].squeeze(),
        "labels": target_tokens["input_ids"].squeeze(),
    }

    return formatted_example

# Tokenize and format the entire dataset
tokenized_dataset = dataset.map(tokenize_and_format_example)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset = tokenized_dataset.remove_columns(dataset["train"].column_names)

In [19]:
input = tokenizer(dataset['train']['input'][1]['sentence1']+dataset['train']['input'][1]['sentence3'], padding="max_length",max_length=32,truncation=True)

In [20]:
tokenizer.decode(input['input_ids'],skip_special_tokens=False)

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 우리는 의자를 큰 책상 주위에 빙 둘러놓았다.학생들이 모두 착석한 뒤 회의가 시작되었다.'

In [37]:
first = '우리는 의자를 큰 책상 주위에 빙 둘러놓았다.'
third = '학생들이 모두 착석한 뒤 회의가 시작되었다.'
target = dataset['train'][1]['output']
input = tokenizer('</s>'+first+'<sep>'+third+'</s>', target,padding="max_length",max_length=32,truncation=True)

In [39]:
tokenizer.decode(input['input_ids'],skip_special_tokens=True)

KeyError: ignored

In [28]:
 input = tokenizer(first, third,target,truncation=True, padding='max_length',max_length=64, return_tensors="pt")

In [36]:
tokenizer.decode(input['labels'][0], skip_special_tokens = False)

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 그러자 의자에 학생들이 하나 둘 앉기 시작했다.'

In [34]:
input['labels'][0]

tensor([    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3, 16256, 48040, 25970,  9402,  9710,
        12462,  6958,  9347, 13872])